In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath

import matplotlib.animation as animation
from matplotlib.ticker import MaxNLocator
from IPython.display import HTML


# Read Data

In [ ]:
def read_guider_data(filename):

    rp = ResourcePath(filename)
    
    with rp.open(mode="rb") as f:
        
        hdu_list = fits.open(f)
        
        N_frames = hdu_list[0].header['N_STAMPS']
        x, y = hdu_list[0].header['ROIROWS'], hdu_list[0].header['ROICOLS']

        imgs = np.zeros((N_frames, x, y))
        timestamps = np.zeros(N_frames)

        if (len(hdu_list) - 1)/2 == N_frames:  ## if has rawstamps
            for i, j in zip(range(N_frames), range(2, len(hdu_list), 2)):
                imgs[i, :, :] = hdu_list[j].data
                timestamps[i] = hdu_list[j].header['STMPTMJD']

        elif len(hdu_list)-1 == N_frames:     ## if no rawstamps
            for i, j in zip(range(N_frames), range(1, len(hdu_list))):
                imgs[i, :, :] = hdu_list[j].data
                timestamps[i] = hdu_list[j].header['STMPTMJD']
    
    grand_hdr = hdu_list[0].header

    return imgs, timestamps, grand_hdr
    

In [ ]:
dayObs = 20250423
seqNum = 139
expId = int(f"{dayObs}{seqNum:05d}")

config = [['R00', 'SG0'], ['R00', 'SG1'], 
          ['R04', 'SG0'], ['R04', 'SG1'], 
          ['R40', 'SG0'], ['R40', 'SG1'],
          ['R44', 'SG0'], ['R44', 'SG1']]

img_list = []
hdr_list = []
time_list = []

for raft, ccd in config:
    filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
    imgs, timestamp, grand_hdr = read_guider_data(filename)    
    
    img_list.append(imgs)
    time_list.append(timestamp)
    hdr_list.append(grand_hdr)


# Check for missing frames

In [ ]:
fig, ax = plt.subplots()

first_stamp = hdr_list[0]['MJD']
for hdr in hdr_list:
    if hdr['MJD'] < first_stamp:
        first_stamp = hdr['MJD']

for i, (raft, ccd) in enumerate(config):

    time_in_seconds = 24*3600*(np.array(time_list[i]) - first_stamp)
    ax.scatter(time_in_seconds, np.zeros(len(time_list[i])) + 8 - i, s=10, label=f'{raft}_{ccd}' )

ax.legend(bbox_to_anchor=(1.0, 1.0))
ax.set_xlabel('Time [s]', fontsize=14)
ax.set_title(f'MC_O_{dayObs}_{seqNum:06d} Guider Frames Timing')

plt.tight_layout()
plt.savefig(f'./Timing/MC_O_{dayObs}_{seqNum:06d}.png')

# Fill-in missing frames with zeros

In [ ]:
time_list_filled = time_list.copy()
img_list_filled = img_list.copy()

N_frames = len(time_list[0])
for i, timestamps in enumerate(time_list):
    if len(timestamps) > N_frames:
        N_frames = len(timestamps)


time_array = np.zeros((8, N_frames))
for i, timestamps in enumerate(time_list):
    time_array[i, :len(timestamps)] = timestamps


missing_frame_ind = []
for j in range(N_frames):
    
    med = np.median(time_array[:,j])
    ind, = np.where(time_array[:,j] != med)
    
    if len(ind) > 0:
        
        for i in ind:
            missing_frame_ind.append([i,j])
            print(med, time_array[i, j])
            filled_stamp = np.insert(time_list_filled[i], j, med)
            time_array[i, :len(filled_stamp)] = filled_stamp
            time_list_filled[i] = filled_stamp

            img_list_filled[i] = np.insert(img_list_filled[i], j, 0, axis=0)


In [ ]:
for (i,j) in missing_frame_ind:
    print(i, j)
    print(img_list_filled[i][j, :,:])

In [ ]:
fig, ax = plt.subplots()

first_stamp = hdr_list[0]['MJD']
for hdr in hdr_list:
    if hdr['MJD'] < first_stamp:
        first_stamp = hdr['MJD']

for i, (raft, ccd) in enumerate(config):

    time_in_seconds = 24*3600*(np.array(time_list_filled[i]) - first_stamp)
    ax.scatter(time_in_seconds, np.zeros(len(time_list_filled[i])) + 8 - i, s=10, label=f'{raft}_{ccd}' )

ax.legend(bbox_to_anchor=(1.0, 1.0))
ax.set_xlabel('Time [s]', fontsize=14)
ax.set_title(f'MC_O_{dayObs}_{seqNum:06d} Guider Frames Timing')

plt.tight_layout()
# plt.savefig(f'./Timing/MC_O_{dayObs}_{seqNum:06d}_filled.png')

# Make plots

In [ ]:
tsp = 24*2600*(time_list_filled[0] - time_list_filled[0][0])


In [ ]:

fig, axes = plt.subplots(2,4, figsize=(8,5))

frame_list = []

for i in range(1, N_frames):

    handlers = []
    for ax, imgs in zip(axes.flatten(), img_list_filled):

        im = ax.imshow(imgs[i, :, 10:], origin='lower', cmap='Greys',  animated=True)
        handlers.append(im)

    frame_list.append(handlers)

for ax, im, (raft, ccd) in zip(axes.flatten(), handlers, config):
    cbar = plt.colorbar(im, ax=ax, location='top', orientation='horizontal', shrink=1.0)
    cbar.locator = MaxNLocator(nbins=3)
    cbar.update_ticks()
    ax.set_xlabel(f'{raft}_{ccd}', fontsize=12)


plt.tight_layout()
    
#ani = animation.FuncAnimation(fig, update, frames=range(1,imgs.shape[0]), interval=120, blit=True, repeat_delay=1000)
ani = animation.ArtistAnimation(fig, frame_list, interval=120, blit=True, repeat_delay=1000)


HTML(ani.to_html5_video())
# ani.save(f'{hdr['OBSID']}_{hdr['RAFTBAY']}_{hdr['CCDSLOT']}.gif')


In [ ]:
ani.save(f'{hdr['OBSID']}_{hdr['RAFTBAY']}_{hdr['CCDSLOT']}.gif')